- [참고 data2vec audio:사용 안하기로함]: https://towhee.io/audio-embedding/data2vec
- [참고 data2vec text]: https://huggingface.co/docs/datasets/nlp_load
- [참고 data2vec load audio*]:https://huggingface.co/docs/datasets/audio_load
- [참고 data2vec embedding]: https://huggingface.co/docs/transformers/model_doc/data2vec#transformers.Data2VecAudioModel
- [참고 handling pytorch dataset]:https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel
- [참고 add data to dataset]: https://pypi.org/project/datasets/

# Embeddings(Text, Audio) Session 1 
- 각세션으로 확대시켜야 함

In [1]:
import torch
from transformers import AutoProcessor, Data2VecAudioModel
from datasets import load_dataset, Dataset, Audio, Features
from glob import glob
import numpy as np
import pandas as pd
from tqdm import tqdm

print(torch.__version__)

/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.1+cu117


In [2]:
import pickle
with open('/home/arplab/project/paradeigma/multi_modal/model/data/paradeigma_KEMDY20_for_embedding.pkl', 'rb') as f:
    KEMDY20_dict = pickle.load(f)

In [3]:
train_sess = KEMDY20_dict['train_df'][0]
val_sess = KEMDY20_dict['val_df'][0]
test_sess = KEMDY20_dict['test_df'][0]


train_df = KEMDY20_dict['train_df'][1]
val_df = KEMDY20_dict['val_df'][1]
test_df = KEMDY20_dict['test_df'][1]


file_lst_train = train_df['Segment ID']
file_lst_val = val_df['Segment ID']
file_lst_test = test_df['Segment ID']

## Audio Embedding

In [4]:
wav_file_lst_train = []
for segment in file_lst_train:
    session = segment[4:6]
    wav_file_lst_train.append(f'/home/arplab/project/paradeigma/multi_modal/org_KEMDy20/Session{session}/' + segment + '.wav')
wav_file_lst_train[:3], len(wav_file_lst_train)

wav_file_lst_val = []
for segment in file_lst_val:
    session = segment[4:6]
    wav_file_lst_val.append(f'/home/arplab/project/paradeigma/multi_modal/org_KEMDy20/Session{session}/' + segment + '.wav')
wav_file_lst_val[:3], len(wav_file_lst_val)

wav_file_lst_test = []
for segment in file_lst_test:
    session = segment[4:6]
    wav_file_lst_test.append(f'/home/arplab/project/paradeigma/multi_modal/org_KEMDy20/Session{session}/' + segment + '.wav')
wav_file_lst_test[:3], len(wav_file_lst_test)

(['/home/arplab/project/paradeigma/multi_modal/org_KEMDy20/Session08/Sess08_script01_User015F_001.wav',
  '/home/arplab/project/paradeigma/multi_modal/org_KEMDy20/Session08/Sess08_script01_User016F_001.wav',
  '/home/arplab/project/paradeigma/multi_modal/org_KEMDy20/Session08/Sess08_script01_User015F_002.wav'],
 2580)

In [6]:
torch.cuda.is_available()

/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [5]:
check_point_wav_model = 'facebook/data2vec-audio-base-960h'
processor = AutoProcessor.from_pretrained(check_point_wav_model)
model_wav_cuda = Data2VecAudioModel.from_pretrained(check_point_wav_model).cuda()
model_wav = Data2VecAudioModel.from_pretrained(check_point_wav_model)

Some weights of the model checkpoint at facebook/data2vec-audio-base-960h were not used when initializing Data2VecAudioModel: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecAudioModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW

### audio embedding method

In [21]:
# padding: https://huggingface.co/docs/transformers/pad_truncation
def wav_embeddings_func(wav_file_lst, processor, model):
    dataset = Dataset.from_dict({"wav": wav_file_lst }).cast_column("wav", Audio())
    wav_array = [dataset[i]['wav']['array'] for i in range(len(wav_file_lst))]
    inputs = processor(wav_array, sampling_rate = sampling_rate, return_tensors = 'pt', padding = 'max_length', max_length= 48000, truncation = True)
    with torch.no_grad():
        output = model_wav(**inputs)
    return output

def make_wav_embedding_pickle(file_lst, processor, model, mode, divisor=300):
    length = len(file_lst)
    quotient = length // divisor
    
    for i in tqdm(range(quotient+1)):
        wav_embeddings = wav_embeddings_func(file_lst[divisor * i: divisor * (i+1)], processor, model)
        wav_embeddings = wav_embeddings['last_hidden_state']
        with open(f'/home/arplab/project/paradeigma/multi_modal/model/data/embedding/paradeigma_KEMDY20_wav_{mode}_embedding_{i+1}.pkl', 'wb') as f:
            pickle.dump(wav_embeddings, f, pickle.HIGHEST_PROTOCOL)
            
        del wav_embeddings
        torch.cuda.empty_cache()
        
def load_wav_embedding_pickle(path_list):
    result = []
    for path in path_list:
        with open(f'/home/arplab/project/paradeigma/multi_modal/model/data/embedding/{path}', 'rb') as f:
            wav_embedding = pickle.load(f)
        result.append(wav_embedding)
        
    return torch.concat(result)

In [104]:
file_lst = [wav_file_lst_train, wav_file_lst_val, wav_file_lst_test]
modes = ['train', 'val','test']
for i in range(3):
    make_wav_embedding_pickle(file_lst[i], processor, model_wav_cuda, modes[i])

100%|██████████| 9/9 [07:51<00:00, 52.37s/it]


In [22]:
import os

dir_path = '/home/arplab/project/paradeigma/multi_modal/model/data/embedding'
file_list = os.listdir(dir_path)
train_path_list = sorted([file for file in file_list if 'KEMDY20_train' in file ])
val_path_list = sorted([file for file in file_list if 'KEMDY20_val' in file ])
test_path_list = sorted([file for file in file_list if 'KEMDY20_test' in file ])

embeddings_wav_train = load_wav_embedding_pickle(train_path_list)
embeddings_wav_val = load_wav_embedding_pickle(val_path_list)
embeddings_wav_test = load_wav_embedding_pickle(test_path_list)

RuntimeError: torch.cat(): expected a non-empty list of Tensors

## Text Embedding
- 참고: https://huggingface.co/docs/datasets/nlp_load
- 참고: https://huggingface.co/docs/transformers/v4.27.2/en/model_doc/data2vec#transformers.Data2VecTextModel
- 참고 한국어 pretrained model: https://huggingface.co/Junmai/KR-Data2VecText-v1

In [5]:
from transformers import PreTrainedTokenizerFast, BartModel
from transformers import AutoModel, AutoTokenizer
import sentencepiece as spm
import pandas as pd
import re


In [6]:
# text data loading
txt_file_lst_train = []
for segment in file_lst_train:
    session = segment[4:6]
    txt_file_lst_train.append(f'/home/arplab/project/paradeigma/multi_modal/org_KEMDy20/Session{session}/' + segment + '.txt')
txt_file_lst_train[:3]

txt_file_lst_val = []
for segment in file_lst_val:
    session = segment[4:6]
    txt_file_lst_val.append(f'/home/arplab/project/paradeigma/multi_modal/org_KEMDy20/Session{session}/' + segment + '.txt')
txt_file_lst_val[:3]

txt_file_lst_test = []
for segment in file_lst_test:
    session = segment[4:6]
    txt_file_lst_test.append(f'/home/arplab/project/paradeigma/multi_modal/org_KEMDy20/Session{session}/' + segment + '.txt')
txt_file_lst_test[:3]

['/home/arplab/project/paradeigma/multi_modal/org_KEMDy20/Session04/Sess04_script01_User008F_010.txt',
 '/home/arplab/project/paradeigma/multi_modal/org_KEMDy20/Session04/Sess04_script01_User007M_009.txt',
 '/home/arplab/project/paradeigma/multi_modal/org_KEMDy20/Session04/Sess04_script01_User007M_010.txt']

In [9]:
checkpoint = 'Junmai/KR-Data2VecText-v1'
model_txt = AutoModel.from_pretrained(checkpoint) 
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model_txt.resize_token_embeddings(len(tokenizer))

Embedding(35002, 768)

### text embedding method

In [23]:
def text_embeddings_func(txt_file_lst, tokenizer, model):
    sentences = []

    for i in txt_file_lst:    
        f = open(i, 'r')
        line = f.readline()
        line = re.sub('\n', '', line)
        line = re.sub('  ', ' ', line)
        line = line.rstrip().lstrip()
        sentences.append(line)
        f.close()
    
    inputs = tokenizer(sentences, padding='max_length', max_length = 80, truncation=True, return_tensors="pt")
    with torch.no_grad():
        txt_embeddings= model_txt(**inputs)
    return txt_embeddings['last_hidden_state']

def make_txt_embedding_pickle(file_lst, tokenizer, model, mode, divisor=300):
    length = len(file_lst)
    quotient = length // divisor
    
    for i in tqdm(range(quotient+1)):
        txt_embeddings = text_embeddings_func(file_lst[divisor * i: divisor * (i+1)], tokenizer, model)
        with open(f'/home/arplab/project/paradeigma/multi_modal/model/data/embedding/paradeigma_KEMDY20_txt_{mode}_embedding_{i+1}.pkl', 'wb') as f:
            pickle.dump(txt_embeddings, f, pickle.HIGHEST_PROTOCOL)
            
        del txt_embeddings
        torch.cuda.empty_cache()
    
def load_txt_embedding_pickle(path_list):
    result = []
    for path in path_list:
        with open(f'/home/arplab/project/paradeigma/multi_modal/model/data/embedding/{path}', 'rb') as f:
            txt_embedding = pickle.load(f)
        result.append(txt_embedding)
        
    return torch.concat(result)

In [16]:
file_lst = [txt_file_lst_train, txt_file_lst_val, txt_file_lst_test]
modes = ['train', 'val','test']
for i in range(3):
    make_txt_embedding_pickle(file_lst[i], tokenizer, model_txt, modes[i])

100%|██████████| 9/9 [02:28<00:00, 16.45s/it]


In [18]:
import os

dir_path = '/home/arplab/project/paradeigma/multi_modal/model/data/embedding'
file_list = os.listdir(dir_path)
train_path_list = sorted([file for file in file_list if 'KEMDY20_txt_train' in file ])
val_path_list = sorted([file for file in file_list if 'KEMDY20_txt_val' in file ])
test_path_list = sorted([file for file in file_list if 'KEMDY20_txt_test' in file ])

embeddings_txt_train = load_txt_embedding_pickle(train_path_list)
embeddings_txt_val = load_txt_embedding_pickle(val_path_list)
embeddings_txt_test = load_txt_embedding_pickle(test_path_list)

# 데이터셋 만들기

In [15]:
# file_name_lst = get_file_names(wav_file_lst)
# file_name_lst[:5]

In [16]:
# # annotation data 가져오기
# df_annotation = pd.read_csv('org_KEMDy20/annotation/' + file_name, skiprows=1)
# df_annotation.Arousal[:3], df_annotation.Emotion[:3], df_annotation.Valence[:3]

In [17]:
def build_dataset(file_name_lst, 
                  text_embeddings, 
                  wav_embeddings, 
                  label_emotion,
                  label_arousal,
                  label_valence, 
                  session_name = 'Session01'):
    dataset = {'file_names': file_name_lst, 
         'text_embeddings': text_embeddings, 
         'wav_embeddings':wav_embeddings,
         'Emotion':label_emotion,
         'Arousal':label_arousal,
         'Valence':label_valence}
    return dataset

In [18]:
session_num_lst = []
annot_num_lst = []
for i in range(40):
    if i <= 8:
        session_num_lst.append('Session0' + str(i+1))
        annot_num_lst.append('Sess0' + str(i+1) + '_eval.csv')
    else:
        session_num_lst.append('Session' + str(i+1))
        annot_num_lst.append('Sess' + str(i+1) + '_eval.csv')
session_num_lst[-5:], annot_num_lst[-5:]

(['Session36', 'Session37', 'Session38', 'Session39', 'Session40'],
 ['Sess36_eval.csv',
  'Sess37_eval.csv',
  'Sess38_eval.csv',
  'Sess39_eval.csv',
  'Sess40_eval.csv'])

In [19]:
dataset = {}
for annot, session in zip(annot_num_lst[3:5], session_num_lst[3:5]):
    
    df_annotation = pd.read_csv('./org_KEMDy20/annotation/' + annot, skiprows=1)
    file_name_lst = df_annotation[' .1']
    
    txt_file_lst = []
    wav_file_lst = []
    for file_name in file_name_lst:
        txt_file_lst.append('./org_KEMDy20/' + session + '/' + file_name + '.txt')
        wav_file_lst.append('./org_KEMDy20/' + session + '/' + file_name + '.wav')
    
    txt_embeddings = text_embeddings_func(txt_file_lst, tokenizer, model_txt)
    wav_embeddings = wav_embeddings_func(wav_file_lst, processor, model_wav_cuda)
    
    dataset[session] = build_dataset(file_name_lst, 
                txt_embeddings, 
                wav_embeddings,  
                df_annotation.Emotion, 
                df_annotation.Arousal, 
                df_annotation.Valence)
    
    torch.cuda.empty_cache()
    print(f'{session}번째 작업이 완료 되었습니다.')

Session04번째 작업이 완료 되었습니다.
Session05번째 작업이 완료 되었습니다.


In [20]:
type(dataset['Session05']['text_embeddings'])

torch.Tensor

In [1]:
# length test
for i in dataset['Session01'].keys():
    print(len(dataset['Session01'][i]), end = ' ')


NameError: name 'dataset' is not defined

# save dataset

In [25]:
import pickle

# save data set
with open('./model/data/lou_dataset_4_5.pkl', 'wb')as f:
    pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)

In [74]:
with open('./model/data/lou_dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)
dataset['Session01'].keys()

dict_keys(['file_names', 'text_embeddings', 'wav_embeddings', 'Emotion', 'Arousal', 'Valence'])